In [20]:
import pandas as pd
import numpy as np
import requests
from requests import get
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup # for web scraping
import seaborn as sns # for beautiful graphs
import scipy.stats as stats # to calculate r^2 for linear regressions
from scipy.stats import powerlaw # for plotting linear regressions
import statsmodels as sm
import matplotlib.ticker as mtick
import re
sns.set()
import warnings
warnings.filterwarnings('ignore')

# Dataset Cleaning 

First , we need to remove the empty columns from the dataset.

In [3]:
#opening the data
data= pd.read_csv('/Users/Mariam/Desktop/chicago-food-inspections/food-inspections.csv',delimiter=',')

# drop all the empty columns
data.drop(['Historical Wards 2003-2015', 'Zip Codes', 'Community Areas','Census Tracts','Wards'], axis=1,inplace=True)

#show the dataframe
display(data.head(3))

,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude,Location
0,2320315,SERENDIPITY CHILDCARE,SERENDIPITY CHILDCARE,2216009.0,Daycare Above and Under 2 Years,Risk 1 (High),1300 W 99TH ST,CHICAGO,IL,60643.0,2019-10-23T00:00:00.000,License Re-Inspection,Pass,NaN,41.714168,-87.655291,"{'longitude': '41.7141680989703', 'latitude': ..."
1,2320342,YOLK TEST KITCHEN,YOLK TEST KITCHEN,2589655.0,Restaurant,Risk 1 (High),1767 N MILWAUKEE AVE,CHICAGO,IL,60647.0,2019-10-23T00:00:00.000,Canvass,Pass w/ Conditions,23. PROPER DATE MARKING AND DISPOSITION - Comm...,41.913588,-87.682203,"{'longitude': '41.9135877900482', 'latitude': ..."
2,2320328,LAS ASADAS MEXICAN GRILL,LAS ASADAS MEXICAN GRILL,2583309.0,Restaurant,Risk 1 (High),3834 W 47TH ST,CHICAGO,IL,60632.0,2019-10-23T00:00:00.000,Canvass,Out of Business,NaN,41.808025,-87.720037,"{'longitude': '41.80802515275297', 'latitude':..."


We need to standardize columns format in order to make it more friendly to use.

In [4]:
#We create a function that fills empty space by '_' and lower case all the letters (reformat all column headers)
def standardize(column):
    column = column.lower().replace(" ", "_")
    column = re.sub('\W+',"", column)
    if len(column) > 1:
        if column[-1] == "_":
            return column[:-1]
    return column

#application of the function to the dataset
data.columns = [standardize(x) for x in data.columns]
display(data.columns)


Index(['inspection_id', 'dba_name', 'aka_name', 'license', 'facility_type',
       'risk', 'address', 'city', 'state', 'zip', 'inspection_date',
       'inspection_type', 'results', 'violations', 'latitude', 'longitude',
       'location'],
      dtype='object')

We need to check if the inspection ID is unique. If it is not the case ,we need to remove the duplicates as an ID reffers to an unique inspection.

In [5]:
#Check is the inspection ID is unique
display(data['inspection_id'].is_unique)

False

In [6]:
#Removing the duplicates 
data.drop_duplicates('inspection_id', inplace=True)

#check if all the duplicates were removed 
display(data['inspection_id'].is_unique)

True

We need to remove all the NA values since we can't use this kind of information. But we need to remove them only from particular column. ( ex : for the violations , NA only means that there were no violations so we need to keep this NA).

In [7]:
#Remove NA values from relevent columns
data.dropna(subset=['inspection_date','license','latitude','longitude'],inplace=True)

We need to remove the time of inspection (useless information) in order to clean the inspection date column

In [8]:
#We remove the time ( all the caracters after 'T')
data['inspection_date']=data['inspection_date'].apply(lambda x : x.split('T')[0])
display(data.head(3))

,inspection_id,dba_name,aka_name,license,facility_type,risk,address,city,state,zip,inspection_date,inspection_type,results,violations,latitude,longitude,location
0,2320315,SERENDIPITY CHILDCARE,SERENDIPITY CHILDCARE,2216009.0,Daycare Above and Under 2 Years,Risk 1 (High),1300 W 99TH ST,CHICAGO,IL,60643.0,2019-10-23,License Re-Inspection,Pass,NaN,41.714168,-87.655291,"{'longitude': '41.7141680989703', 'latitude': ..."
1,2320342,YOLK TEST KITCHEN,YOLK TEST KITCHEN,2589655.0,Restaurant,Risk 1 (High),1767 N MILWAUKEE AVE,CHICAGO,IL,60647.0,2019-10-23,Canvass,Pass w/ Conditions,23. PROPER DATE MARKING AND DISPOSITION - Comm...,41.913588,-87.682203,"{'longitude': '41.9135877900482', 'latitude': ..."
2,2320328,LAS ASADAS MEXICAN GRILL,LAS ASADAS MEXICAN GRILL,2583309.0,Restaurant,Risk 1 (High),3834 W 47TH ST,CHICAGO,IL,60632.0,2019-10-23,Canvass,Out of Business,NaN,41.808025,-87.720037,"{'longitude': '41.80802515275297', 'latitude':..."


We need to check if there are only information from Chicago.

In [9]:
#check if we have only data from chicage, if not we need to remove all the extra information
data.city.unique()

array(['CHICAGO', nan, 'Chicago', 'CCHICAGO', 'CHICAGO.',
       'CHESTNUT STREET', 'CHICAGOCHICAGO', 'chicago', 'CHICAGOHICAGO',
       'CHicago', '312CHICAGO', 'BEDFORD PARK', 'CHCICAGO',
       'CHARLES A HAYES', 'CHCHICAGO', 'CHICAGOI', 'SUMMIT', 'WESTMONT',
       'LOMBARD', 'INACTIVE', 'alsip', 'BLUE ISLAND'], dtype=object)

We need to remove :   Bedford Park (Gas Station) ; Blue Island; Lombard ( a village near to Chicago) , Summit ( a city near to Chicago) ; WESTMONT ( village near Chicago) ; aslip (suburb of chicago)
We need to replace by chicago : 'CHARLES A HAYES'  (postal location), 312Chicago (Restaurant) ; CHICAGOI (Chicago)  ; CHESTNUT STREET (street in chicago); INACTIVE (out of business restaurant in Chicago); Chestnut street ;

In [10]:
#Check if the state is unique
display(data.state.unique())

#As the state is unique and we will not use this columns for our further investigations, we can drop it 
data.drop(['state'], axis=1,inplace=True)

#Check if there are other city than Chicago
display(data.inspection_id.groupby(data['city']).count())

#Drop the selected locations 
data = data[~data['city'].isin(["BEDFORD PARK", "BLUE ISLAND", "LOMBARD","SUMMIT","WESTMONT","alsip"])]

#check if the column is clean 
display(data.inspection_id.groupby(data['city']).count())

#now that we are sure that we have only information from Chicago ,we can delete the city columns
data.drop(['city'], axis=1,inplace=True)

array(['IL', nan], dtype=object)

city
312CHICAGO              2
BEDFORD PARK            2
BLUE ISLAND             1
CCHICAGO               45
CHARLES A HAYES         4
CHCHICAGO               6
CHCICAGO                3
CHESTNUT STREET        11
CHICAGO            193192
CHICAGO.                2
CHICAGOCHICAGO          7
CHICAGOHICAGO           2
CHICAGOI                3
CHicago                12
Chicago               317
INACTIVE                8
LOMBARD                 1
SUMMIT                  4
WESTMONT                1
alsip                   1
chicago                82
Name: inspection_id, dtype: int64

city
312CHICAGO              2
CCHICAGO               45
CHARLES A HAYES         4
CHCHICAGO               6
CHCICAGO                3
CHESTNUT STREET        11
CHICAGO            193192
CHICAGO.                2
CHICAGOCHICAGO          7
CHICAGOHICAGO           2
CHICAGOI                3
CHicago                12
Chicago               317
INACTIVE                8
chicago                82
Name: inspection_id, dtype: int64

If we explore the license numbers, we find that there are some null license number. We need to remove them,

In [11]:
# Drop "0.0" licenses
data = data[data.license != 0.0]

In [17]:
#looking for the inspection results type
data.groupby('results')['inspection_id'].count()

results
Business Not Located        49
Fail                     37354
No Entry                  6158
Not Ready                 1843
Out of Business          16676
Pass                    104679
Pass w/ Conditions       26566
Name: inspection_id, dtype: int64

We are only interested in 3 type of results : Pass , Pass with condition and Fail . We need to remove the others.

In [18]:
# Only consider successful inspections
data = data[~data.results.isin(['Out of Business', 'Business Not Located', 'No Entry','Not Ready'])]

In [21]:
#Converting inspection dates into float numbers
data.inspection_date = data.inspection_date.astype('datetime64[ns]')

#make sure that the latitude and longitude are float numbers
data['latitude'] = data['latitude'].astype(float)
data['longitude'] = data['longitude'].astype(float)

#convert into string (useful for further steps ) -
data.zip=data.zip.astype(str)

#reformat the zip code writing in order to compare it with the zip code in geojson file (vizualisation step)
data['zip']=data['zip'].apply(lambda x : x.split('.')[0])



For a coordination purpose , we will export this clean dataset in a csv file.

In [27]:
#export the dataframe to a csv 
data.to_csv(r'./clean_dataset.csv')

# Dataset with only the last inspection results 

In [22]:
#we want to selection only the informations from the last inspection for each facility
date_last_inspection=pd.DataFrame(data.groupby('license')['inspection_date'].max())
date_last_inspection
#by merging 'data' and 'date_last_inspection' we can create a data frame containing only infromations about the last inspection for each facility
data_last_inspection=pd.merge(date_last_inspection,data, on=['license', 'inspection_date'])
data_last_inspection.groupby('license').count()
data_last_inspection.rename(columns={'inspection_date': 'last_inspection_date'}, inplace=True)
data_last_inspection.head(3)

,license,last_inspection_date,inspection_id,dba_name,aka_name,facility_type,risk,address,zip,inspection_type,results,violations,latitude,longitude,location
0,1.0,2010-06-04,250567,HARVEST CRUSADES MINISTRIES,HARVEST CRUSADES MINISTRIES,Special Event,Risk 2 (Medium),118 N CENTRAL AVE,60644,Special Events (Festivals),Pass,NaN,41.882845,-87.765095,"{'longitude': '41.88284507471884', 'latitude':..."
1,2.0,2018-02-13,2144871,COSI,COSI,Restaurant,Risk 1 (High),230 W MONROE ST,60606,Canvass,Pass w/ Conditions,3. POTENTIALLY HAZARDOUS FOOD MEETS TEMPERATUR...,41.880757,-87.634709,"{'longitude': '41.88075715864721', 'latitude':..."
2,9.0,2019-08-09,2304407,XANDO COFFEE & BAR / COSI SANDWICH BAR,XANDO COFFEE & BAR / COSI SANDWICH BAR,Restaurant,Risk 1 (High),116 S MICHIGAN AVE,60603,Canvass,Pass w/ Conditions,"1. PERSON IN CHARGE PRESENT, DEMONSTRATES KNOW...",41.880396,-87.624502,"{'longitude': '41.88039583825962', 'latitude':..."


# Violations dataframe

We want to make the violation column more readible.
Using the followingl link https://webapps1.chicago.gov/healthinspection/Code_Violations.jsp#minor  , each number is associated with a unique violation.
For simplification purpose, we will consider only the violations from 1 to 44 and the violation number 70. 

In [28]:
#function that split the violations number from the comments

def violation_separator(violations):
    violation_number = pd.Series([])   #creating an empty dataframe in order to stock the violation numbers
    if type(violations) == str:
        violations = violations.split(' | ') #each different violation is separated by a ' | ' in a dataframe cell
        for violation in violations:        #now, we can iterate on the differente violations of each inspection
            index = "#" + violation.split('.')[0]  #the index refers to the violation number
            violation_number[index] = 1 #add 1 if there is a violation #.. and 0 if not.
    return violation_number

#we create a new data frame by applying the function to the dataset thhe old one and fill the nan value by 0 . 1= violation , 0= no violation
violations_data = data_last_inspection.violations.apply(violation_separator).fillna(0)

In [29]:
# Generate column names
critical = [("#" + str(num)) for num in range(1, 15)]
serious = [("#" + str(num)) for num in range(15, 30)]
minor = [("#" + str(num)) for num in range(30, 45)]
minor.append("#70")

# Create complete list of column names
columns = critical + serious + minor

# Create dataframe using column names, violation data and inspection ID
violations_data = pd.DataFrame(violations_data, columns=columns)
violations_data['inspection_id'] = data_last_inspection.inspection_id
violations_data['license'] = data_last_inspection.license

In [30]:
#counting the kind of violation per inspection ID
violation_counts = pd.DataFrame({
    "critical_count": violations_data[critical].sum(axis=1),
    "serious_count": violations_data[serious].sum(axis=1),
    "minor_count": violations_data[minor].sum(axis=1)
})

violation_counts['inspection_id'] = data_last_inspection.inspection_id
violation_counts['license'] = data_last_inspection.license
# Display selection of sums dataframe
violation_counts.iloc[3:6]

,critical_count,serious_count,minor_count,inspection_id,license
3,3.0,0.0,1.0,2222357,40.0
4,0.0,0.0,2.0,2145199,62.0
5,1.0,0.0,3.0,1138213,85.0


In [31]:
#minor violatios are more frequent .
violation_counts.sum()

critical_count    1.511000e+04
serious_count     6.942000e+03
minor_count       4.783300e+04
inspection_id     5.325937e+10
license           6.007050e+10
dtype: float64